In [159]:
from elasticsearch import Elasticsearch, helpers
from datetime import datetime,time,timedelta
import pandas as pd

In [5]:
es = Elasticsearch("http://localhost:9400",basic_auth=("elastic","secret"),verify_certs=False)
es.indices.get_alias(index="*")

ObjectApiResponse({'education': {'aliases': {}}, 'data_description': {'aliases': {}}, 'achievements': {'aliases': {}}, 'activity': {'aliases': {}}, 'log': {'aliases': {}}, 'appsettings': {'aliases': {}}, 'exercise': {'aliases': {}}, 'baseline': {'aliases': {}}, 'plan': {'aliases': {}}, 'account': {'aliases': {}}})

In [131]:
#engang
def update_goal(userid,goalid,progress=1,achievedat=1):
      #possible goal ids
      #SetGoalSetting,SetSleepTool,UseMindfulness,QACompleted,SessionCompleted
      res = es.get(index="achievements", 
            id=userid+"_"+goalid)
      id=res["_id"]
      res=res["_source"]
      res["progress"]=progress
      res["achievedat"]=achievedat
      es.update(index='achievements',id=id,
                  doc=res)


In [17]:
def complete_quiz(userid):
    correct=es.search(index="education",query={"bool":
                                          {"must":[
                                              {"match":{"is_correct":True}},
                                                {"match":{"userid":"carmela"}}
                                          ]
                                         
                                          }}
    ,size=900).body["hits"]["hits"]
    num_correct=len(correct)
    update_goal(userid,"EducationalQuizAnswers",num_correct,float(num_correct>=1))
    for i in [1,2,7,14,25]:
        completed=-1
        if num_correct>=i:
          completed=1
        update_goal(userid,"EducationalCorrectQuiz"+str(i),num_correct,completed)


In [18]:
def complete_educational_read(userid):
    read=es.search(index="education",query={"match":{"userid":userid}},size=900).body["hits"]["hits"]
    num_read=len(read)
    update_goal(userid,"EducationalRead",num_read,float(num_read>=1))
    for i in [1,3,7,14,25,40,60,80,100]:
        completed=-1
        if num_read>=i:
          completed=1
        update_goal(userid,"EducationalMaterialRead"+str(i),num_read,completed)


In [19]:
complete_educational_read("thea")

{'_index': 'achievements', '_id': 'thea_EducationalRead', '_version': 1, '_seq_no': 70, '_primary_term': 1, 'found': True, '_source': {'userid': 'thea', 'achievementid': 'EducationalRead', 'progress': 0, 'goal': -1, 'achievedat': -1}}
{'_index': 'achievements', '_id': 'thea_EducationalMaterialRead1', '_version': 2, '_seq_no': 74, '_primary_term': 1, 'found': True, '_source': {'userid': 'thea', 'achievementid': 'EducationalMaterialRead1', 'progress': 1, 'goal': 1, 'achievedat': 1}}
{'_index': 'achievements', '_id': 'thea_EducationalMaterialRead3', '_version': 1, '_seq_no': 11, '_primary_term': 1, 'found': True, '_source': {'userid': 'thea', 'achievementid': 'EducationalMaterialRead3', 'progress': 0, 'goal': 3, 'achievedat': -1}}
{'_index': 'achievements', '_id': 'thea_EducationalMaterialRead7', '_version': 1, '_seq_no': 12, '_primary_term': 1, 'found': True, '_source': {'userid': 'thea', 'achievementid': 'EducationalMaterialRead7', 'progress': 0, 'goal': 7, 'achievedat': -1}}
{'_index':

In [21]:
def total_steps(userid):
    res = es.search(index="activity", query={'match' : {"userid":userid}},size=10000)["hits"]["hits"]
    steps=list(map(lambda x: x["_source"]["steps"],res))
    total_steps=sum(steps)
    for i in [42.195,50,100,200,400,600,800,1000]:
        complete=-1
        if total_steps>=i*1000:
            complete=1
        if i == 42.195:
            update_goal(userid,"Marathon",total_steps,complete)
        else:
            update_goal(userid,"TotalSteps"+str(i),total_steps,complete)


In [22]:
total_steps("thea")

In [167]:
def avg_weekly_steps(userid):
    steps = es.search(index="activity", query={"bool":{"must":[
                                                  {'match' : {"userid":userid}},
                                                  {'range' : {"start":{"gte":(datetime.now()- timedelta(weeks=1)).timestamp()*1000}}},
                                                    ]
                                                  }
                                          }
                      ,size=10000)["hits"]["hits"]
    steps=list(map(lambda x: x["_source"]["steps"],steps))
    for i in [5,6,8,10,12]:
        complete=-1
        if sum(steps)/7>=i*1000:
          complete=1
          update_goal(userid,"AverageStepsWeek"+str(i),round(sum(steps)/7),complete)


In [168]:
avg_weekly_steps("thea")

In [155]:
def daily_steps(userid):
    current_goal = es.search(index="plan", query={'match' : {"userid":userid}},sort=[
    {
      "start": {
        "order": "desc"
      }
    }],size=10000)["hits"]["hits"][0]["_source"]["plan"]["activity"]["goal"]
    steps = es.search(index="activity", query={"bool":{"must":[
                                                  {'match' : {"userid":userid}},
                                                  {'range' : {"start":{"gte":datetime.combine(datetime.now(), time.min).timestamp()*1000}}},
                                                  {'range' : {"start":{"lte":datetime.combine(datetime.now(), time.max).timestamp()*1000}}},
                                                    ]
                                                  }
                                          }
                      ,size=10000)["hits"]["hits"]
    steps=list(map(lambda x: x["_source"]["steps"],steps))
    #checks if goal that is currently active is reached
    first_ach=es.get(index="achievements", id=userid+"_"+"DailyGoalSteps1")["_source"]
    last_updated= first_ach["achievedat"]
    progress=first_ach["progress"]
    print(sum(steps))
    for i in [10,12,14,16,18,20]:
        completed=-1
        if sum(steps)>=i*1000:
            completed=1
        update_goal(userid,"DailyRecordSteps"+str(i),sum(steps),completed)
    if last_updated<datetime.now().timestamp()*1000:
      if sum(steps)>=current_goal:
          for i in [1,3,7,14,25,40,60,80,100]:
            if i==1:
              update_goal(userid,"DailyGoalSteps"+str(i),progress+1,datetime.combine(datetime.now(), time.max).timestamp()*1000)
            else:
               update_goal(userid,"DailyGoalSteps"+str(i),progress+1,1 if progress+1>=i else -1)
          ach=es.get(index="achievements", id=userid+"_"+"RowCompletedSteps3")["_source"]
          last_updated= ach["achievedat"]
          progress=ach["progress"]
          #if the last time it was updated was yesterday then increase counter by one, if not set to 1
          if last_updated>(datetime.combine(datetime.now(), time.min)-timedelta(days=1)).timestamp()*1000:
            for i in [3,7,14,25,40]:
              update_goal(userid,"RowCompletedSteps"+str(i),progress+1,1 if progress+1>=i else -1)
          else:
             update_goal(userid,"RowCompletedSteps"+str(i),1,-1)
          

In [156]:
daily_steps("thea")

4500


In [67]:
def completed_exercises(userid):
    res = es.search(index="plan", query={'match' : {"userid":userid}},size=10000)["hits"]["hits"]
    exercises=list(map(lambda x: x["_source"],res))
    display(exercises[0])
    # exercises=pd.DataFrame(exercises)
    # exercises.date=exercises.date.apply(lambda x: datetime.fromtimestamp(x).date())

    # total_exercises=sum(exercises)
    # for i in [1,3,7,14,25,40,60,80,100]:
    #     complete=-1
    #     if total_exercises>=i:
    #         complete=1
    #     update_goal("thea","TotalExercises"+str(i),total_exercises,complete)

In [68]:
# es.search(index="exercise", query={'match' : {"userid":"thea"}},size=10000)["hits"]["hits"]

In [75]:
completed_exercises("thea") 

{'userid': 'thea',
 'created': 1725460035,
 'start': '2024-09-04T16:27:15.586844',
 'end': '2024-09-11T23:59:59.999999',
 'exercises_duration': 15,
 'history': [],
 'plan': {'date': '2024-09-04T16:27:15.586844',
  'educations': [{'is_quiz': False,
    'educationid': 'Barrier family work_2',
    'used': False,
    'thisweek': False,
    'priority': 4,
    'canbequiz': False,
    'expiry_weeks': 8,
    'group': 38},
   {'is_quiz': False,
    'educationid': 'Goal setting_3',
    'used': False,
    'thisweek': False,
    'priority': 4,
    'canbequiz': False,
    'expiry_weeks': 2,
    'group': 27},
   {'is_quiz': False,
    'educationid': 'FA Reassurance_5',
    'used': False,
    'thisweek': False,
    'priority': 4,
    'canbequiz': False,
    'expiry_weeks': 1,
    'group': 21},
   {'is_quiz': False,
    'educationid': 'Guideline LBP_1',
    'used': False,
    'thisweek': False,
    'priority': 4,
    'canbequiz': True,
    'expiry_weeks': 1,
    'group': 1},
   {'is_quiz': False,
    

### OLD

In [ ]:
#gjennomfør alle 3 aktivitetene(antall ganger)
#1,3,7,14,25,40

In [ ]:
#korrekt svar på quiz(antall ganger)
#1,3,7,14,25

In [ ]:

#engang


def total_steps(userid):
    #This can be implemented more efficiently with fewer requests
    completed_achievements=[]
    res = es.search(index="activity", query={'match' : {"userid":userid}},size=10000)["hits"]["hits"]
    res=list(map(lambda x: x["_source"],res))
    df=pd.DataFrame(res)
    if len(df)==0:
        return None
    df.start=df.start.apply(lambda x: datetime.fromtimestamp(x/1000))
    df.end=df.end.apply(lambda x: datetime.fromtimestamp(x/1000))
    df
    for i in [50,100,200,400,600,800,1000,42.195]:
        if i!=42.195:
            ach_id="TotalSteps"+str(i)
        else:
            ach_id="Marathon"
        res = es.search(index="achievements", 
            query={"bool":{"must":[
                {'match' : 
                    {"userid":userid}},
                {'match' : 
                    {"achievementid":ach_id}}
            ]}},size=10000)["hits"]["hits"][0]
        id=res["_id"]
        res=res["_source"]
        if ach_id=="Marathon":
            res["progress"]=0
        else:
            res["progress"]=df.steps.sum()
        if df.steps.sum()>=i*1000:
            res["achievedat"]=1
            es.update(index='achievements',id=id,doc=res)
        else:
            break
        

def avg_steps_per_week(userid):
    #This can be implemented more efficiently with fewer requests
    res = es.search(index="activity", query={'match' : {"userid":userid}},size=10000)["hits"]["hits"]
    res=list(map(lambda x: x["_source"],res))
    df=pd.DataFrame(res)
    df.start=df.start.apply(lambda x: datetime.fromtimestamp(x/1000))
    df.end=df.end.apply(lambda x: datetime.fromtimestamp(x/1000))
    # display(df[df.start > datetime.now() - pd.to_timedelta("7day")])
    for i in [5,6,8,10,12]:
        res = es.search(index="achievements", 
            query={"bool":{"must":[
                {'match' : 
                    {"userid":userid}},
                {'match' : 
                    {"achievementid":"AverageStepsWeek"+str(i)}}
            ]}},size=10000)["hits"]["hits"][0]
        id=res["_id"]
        res=res["_source"]
        average=df.steps.sum()/7
        res["progress"]=average
        if average>=i*1000:
            res["achievedat"]=1
            es.update(index='achievements',id=id,doc=res)
        else:
            break



def steps_in_a_day(userid):
    res = es.search(index="activity", query={'match' : {"userid":userid}},size=10000)["hits"]["hits"]
    res=list(map(lambda x: x["_source"],res))
    df=pd.DataFrame(res)
    df.start=df.start.apply(lambda x: datetime.fromtimestamp(x/1000))
    df.end=df.end.apply(lambda x: datetime.fromtimestamp(x/1000))
    max_steps_in_a_day=df.groupby(df.start.dt.date).steps.sum().max()
    for i in [10,12,14,16,18,20]:
        res = es.search(index="achievements", 
            query={"bool":{"must":[
                {'match' : 
                    {"userid":userid}},
                {'match' : 
                    {"achievementid":"DailyRecordSteps"+str(i)}}
            ]}},size=10000)["hits"]["hits"][0]
        id=res["_id"]
        res=res["_source"]
        res["progress"]=max_steps_in_a_day
        if max_steps_in_a_day>=i*1000:
            res["achievedat"]=1
            es.update(index='achievements',id=id,doc=res)
        else:
            break
#fullfør en tilpasning av planen din
#fullfør intro


# def correct_quizes()

In [ ]:
res_achievements = es.search(index="achievements", query={'match' : {"userid":"user"}},size=10000)["hits"]["hits"][0]

In [ ]:
completed_achievements=[]
res = es.search(index="activity", query={'match' : {"userid":"user"}},size=10000)["hits"]["hits"]
res=list(map(lambda x: x["_source"],res))
df=pd.DataFrame(res)
df.start=df.start.apply(lambda x: datetime.fromtimestamp(x/1000))
df.end=df.end.apply(lambda x: datetime.fromtimestamp(x/1000))
df

,start,end,type,steps,userid
0,2024-07-21 13:40:00,2024-07-21 14:28:00,current,14000,user
1,2024-07-15 10:00:00,2024-07-15 11:00:00,walking,200,user
2,2024-07-16 11:00:00,2024-07-16 12:00:00,walking,196,user
3,2024-07-18 10:00:00,2024-07-18 11:00:00,walking,2000,user
4,2024-07-18 03:00:00,2024-07-18 04:00:00,walking,400,user
5,2024-07-19 11:00:00,2024-07-19 12:00:00,walking,5679,user
6,2024-07-20 15:00:00,2024-07-20 16:00:00,walking,2000,user
7,2024-07-21 14:00:00,2024-07-21 15:00:00,walking,6000,user
8,2024-07-21 13:00:00,2024-07-21 14:00:00,walking,8000,user
9,2024-08-12 15:01:00,2024-08-12 15:02:00,current,102000,user


In [ ]:
#max steps per day
def max_steps_per_day(userid,df):
    steps_per_day=df.groupby(df.start.dt.floor('1D')).steps.sum()
    for i in [10,12,14,16,18,20]:
        res = es.search(index="achievements", 
            query={"bool":{"must":[
                {'match' : 
                    {"userid":userid}},
                {'match' : 
                    {"achievementid":"DailyRecordSteps"+str(i)}}
            ]}},size=10000)["hits"]["hits"][0]
        id=res["_id"]
        res=res["_source"]
        res["progress"]=steps_per_day.max()
        if steps_per_day.max()>=i*1000:
            res["achievedat"]=1
        es.update(index='achievements',id=id,doc=res)

In [ ]:
def total_steps(userid):
    completed_achievements=[]
    res = es.search(index="activity", query={'match' : {"userid":userid}},size=10000)["hits"]["hits"]
    res=list(map(lambda x: x["_source"],res))
    df=pd.DataFrame(res)
    df.start=df.start.apply(lambda x: datetime.fromtimestamp(x/1000))
    df.end=df.end.apply(lambda x: datetime.fromtimestamp(x/1000))
    df
    display(df)
    for i in [50,100,200,400,600,800,1000,42.195]:
        if i!=42.195:
            ach_id="TotalSteps"+str(i)
        else:
            ach_id="Marathon"
        res = es.search(index="achievements", 
            query={"bool":{"must":[
                {'match' : 
                    {"userid":userid}},
                {'match' : 
                    {"achievementid":ach_id}}
            ]}},size=10000)["hits"]["hits"][0]
        print(res)
        id=res["_id"]
        res=res["_source"]
        if ach_id=="Marathon":
            res["progress"]=0
        else:
            res["progress"]=df.steps.sum()
            print(df.steps.sum())
        if df.steps.sum()>=i*1000:
            res["achievedat"]=1
        print(res)
        es.update(index='achievements',id=id,doc=res)

In [ ]:
total_steps("user")

In [ ]:
def avg_steps_per_week(userid):
    res = es.search(index="activity", query={'match' : {"userid":userid}},size=10000)["hits"]["hits"]
    res=list(map(lambda x: x["_source"],res))
    df=pd.DataFrame(res)
    df.start=df.start.apply(lambda x: datetime.fromtimestamp(x/1000))
    df.end=df.end.apply(lambda x: datetime.fromtimestamp(x/1000))
    # display(df[df.start > datetime.now() - pd.to_timedelta("7day")])
    for i in [5,6,8,10,12]:
        res = es.search(index="achievements", 
            query={"bool":{"must":[
                {'match' : 
                    {"userid":userid}},
                {'match' : 
                    {"achievementid":"AverageStepsWeek"+str(i)}}
            ]}},size=10000)["hits"]["hits"][0]
        id=res["_id"]
        res=res["_source"]
        average=df.steps.sum()/7
        res["progress"]=average
        if average>=i*1000:
            res["achievedat"]=1
        es.update(index='achievements',id=id,doc=res)

In [ ]:
avg_steps_per_week("user")

In [ ]:
def steps_in_a_day(userid):
    res = es.search(index="activity", query={'match' : {"userid":userid}},size=10000)["hits"]["hits"]
    res=list(map(lambda x: x["_source"],res))
    df=pd.DataFrame(res)
    df.start=df.start.apply(lambda x: datetime.fromtimestamp(x/1000))
    df.end=df.end.apply(lambda x: datetime.fromtimestamp(x/1000))
    max_steps_in_a_day=df.groupby(df.start.dt.date).steps.sum().max()
    for i in [10,12,14,16,18,20]:
        res = es.search(index="achievements", 
            query={"bool":{"must":[
                {'match' : 
                    {"userid":userid}},
                {'match' : 
                    {"achievementid":"DailyRecordSteps"+str(i)}}
            ]}},size=10000)["hits"]["hits"][0]
        id=res["_id"]
        res=res["_source"]
        res["progress"]=max_steps_in_a_day
        if max_steps_in_a_day>=i*1000:
            res["achievedat"]=1
            es.update(index='achievements',id=id,doc=res)
        else:
            break

In [ ]:
steps_in_a_day("user")

{'_index': 'achievements', '_id': '2Ig21ZABlNYZt4JGnoYK', '_score': 3.4720056, '_source': {'userid': 'user', 'achievementid': 'DailyRecordSteps10', 'progress': 204000, 'goal': 10000, 'achievedat': 1}}
204000
{'_index': 'achievements', '_id': '2Yg21ZABlNYZt4JGnoYK', '_score': 3.4720056, '_source': {'userid': 'user', 'achievementid': 'DailyRecordSteps12', 'progress': 204000, 'goal': 12000, 'achievedat': 1}}
204000
{'_index': 'achievements', '_id': '2og21ZABlNYZt4JGnoYK', '_score': 3.4720056, '_source': {'userid': 'user', 'achievementid': 'DailyRecordSteps14', 'progress': 204000, 'goal': 14000, 'achievedat': 1}}
204000
{'_index': 'achievements', '_id': '24g21ZABlNYZt4JGnoYK', '_score': 3.4720056, '_source': {'userid': 'user', 'achievementid': 'DailyRecordSteps16', 'progress': 204000, 'goal': 16000, 'achievedat': 1}}
204000
{'_index': 'achievements', '_id': '3Ig21ZABlNYZt4JGnoYK', '_score': 3.4720056, '_source': {'userid': 'user', 'achievementid': 'DailyRecordSteps18', 'progress': 204000, 

In [ ]:
def daily_goal_met(userid):
    res = es.search(index="activity", query={'match' : {"userid":userid}},size=10000)["hits"]["hits"]
    res=list(map(lambda x: x["_source"],res))
    df=pd.DataFrame(res)
    df.start=df.start.apply(lambda x: datetime.fromtimestamp(x/1000))
    df.end=df.end.apply(lambda x: datetime.fromtimestamp(x/1000))
    sums_days=df.groupby(df.start.dt.date).steps.sum()
    goals=list(map(lambda x:{"start":x["_source"]["start"],"end":x["_source"]["end"],"goal":x["_source"]["plan"]["activity"]["goal"]},es.search(index="plan", query={'match' : {"userid":"user"}},sort=[
    {
      "start": {
        "order": "desc"
      }
    }
  ],size=10000)["hits"]["hits"]))#[-1]
    display(df)
    print(goals)
    # num_reached=0
    # for goal in goals:
    #     temp_df=df.loc[(df.start>goal["start"]) & (df.end<goal["end"])]
    #     if temp_df.steps.sum()>=goal["goal"]:
    #         num_reached+=1

    # for i in [1,3,7,14,25,40,60,80,100]:
    #     res = es.search(index="achievements", 
    #             query={"bool":{"must":[
    #                 {'match' : 
    #                     {"userid":userid}},
    #                 {'match' : 
    #                     {"achievementid":"DailyGoalSteps"+str(i)}}
    #             ]}},size=10000)["hits"]["hits"][0]
    #     id=res["_id"]
    #     res=res["_source"]
    #     res["progress"]=num_reached
    #     if num_reached>=i:
    #         res["achievedat"]=1
    #     print(res)
        # es.update(index='achievements',id=id,doc=res)

In [ ]:
daily_goal_met("user")

,start,end,type,steps,userid
0,2024-07-21 13:40:00,2024-07-21 14:28:00,current,14000,user
1,2024-07-15 10:00:00,2024-07-15 11:00:00,walking,200,user
2,2024-07-16 11:00:00,2024-07-16 12:00:00,walking,196,user
3,2024-07-18 10:00:00,2024-07-18 11:00:00,walking,2000,user
4,2024-07-18 03:00:00,2024-07-18 04:00:00,walking,400,user
5,2024-07-19 11:00:00,2024-07-19 12:00:00,walking,5679,user
6,2024-07-20 15:00:00,2024-07-20 16:00:00,walking,2000,user
7,2024-07-21 14:00:00,2024-07-21 15:00:00,walking,6000,user
8,2024-07-21 13:00:00,2024-07-21 14:00:00,walking,8000,user
9,2024-08-12 15:01:00,2024-08-12 15:02:00,current,102000,user


[{'start': '2024-08-12T11:55:24.206245', 'end': '2024-08-19T23:59:59.999999', 'goal': 3000}]


In [ ]:
#nå daglig mål for skritt(på rad) + for en hel uke
#3,7,14,25,40

In [ ]:
goals=list(map(lambda x:{"start":x["_source"]["start"],"end":x["_source"]["end"],"goal":x["_source"]["plan"]["activity"]["goal"]},es.search(index="plan", query={'match' : {"userid":"user"}},size=10000)["hits"]["hits"]))#[-1]



reached=True
days_in_a_row=0
initial_date=datetime.now().date()
while reached:


{'start': '2024-07-21T11:43:50.657581',
 'end': '2024-07-28T23:59:59.999999',
 'goal': 3000}

In [ ]:
#gjennomsnittlig antall skritt på en uke
#5,6,8,10,12